## Importing Libraries

In [1]:
from transformers import pipeline, AutoTokenizer, TFAutoModelForSeq2SeqLM
from bs4 import BeautifulSoup
import tf_keras as keras
import re
import requests

## Initializing Tokenizer, Summarizer and Sentiment Analysis models

In [ ]:
# Initialize the tokenizer for the financial summarization model
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

# Initialize the summarization model
summarizer = TFAutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

# Initialize the sentiment analysis pipeline using the FinBERT model
sentiment_analysis = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

In [ ]:
# Initialize the tickers to be used in the sentiment analysis
# Using the top 2 cryptocurrencies as an example
tickers = ['BTC', 'ETH']

## Search news URL method

In [ ]:
# Function to find news articles for a given ticker
def search_for_news_urls(ticker):
    # Construct the Google search URL for Yahoo Finance news related to the ticker
    search_url = f"https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws"
    
    # Make a request to the search URL
    response = requests.get(search_url)
    
    # Parse the HTML content of the response
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all anchor tags in the parsed HTML content
    anchor_tags = soup.find_all('a')
    
    # Extract the href attributes from the anchor tags
    hrefs = [link['href'] for link in anchor_tags]
    
    return hrefs

In [ ]:
# Create a dictionary where each ticker is mapped to its corresponding news article URLs
# The search_for_news_urls function is called for each ticker to get the news URLs
raw = {ticker: search_for_news_urls(ticker) for ticker in tickers}

# Display the keys of the raw dictionary, which are the tickers
raw.keys()